In [1]:
import requests
import pandas as pd
from sqlalchemy import create_engine
import sqlite3

In [15]:
# Step 1: Extract Data from API in Chunks
def fetch_data_in_chunks(api_url, params, start, end, chunk_size=1000):
    for i in range(start, end, chunk_size):
        params['start'] = i
        params['end'] = i + chunk_size
        response = requests.get(api_url, params=params)
        if response.status_code == 200:
            data = response.json()
            df = pd.json_normalize(data)
            yield df
        else:
            print(f"Error fetching data for chunk {i} to {i + chunk_size}: {response.status_code}")
            continue


In [20]:
# API details
api_url = "https://data.colorado.gov/resource/4ykn-tg5h.json" 
params = {
    'apikey': "gxc0jsptwdopcqxffpl21c8b", 
    'start': 0,
    'end': 1000
}

# Database engine
engine = create_engine('sqlite:///weather_data.db')

# Define the range and chunk size
start = 0
end = 10000  # Replace with the actual number of records you need to process
chunk_size = 1000


In [19]:

# Step 2: Load Data into Database in Chunks
for df_chunk in fetch_data_in_chunks(api_url, params, start, end, chunk_size):
    df_chunk.to_sql('weather', con=engine, if_exists='append', index=False)
    print(f"Data chunk {params['start']} to {params['end']} loaded into the database successfully!")


Error fetching data for chunk 0 to 1000: 400
Error fetching data for chunk 1000 to 2000: 400
Error fetching data for chunk 2000 to 3000: 400
Error fetching data for chunk 3000 to 4000: 400
Error fetching data for chunk 4000 to 5000: 400
Error fetching data for chunk 5000 to 6000: 400
Error fetching data for chunk 6000 to 7000: 400
Error fetching data for chunk 7000 to 8000: 400
Error fetching data for chunk 8000 to 9000: 400
Error fetching data for chunk 9000 to 10000: 400
Error fetching data for chunk 10000 to 11000: 400
Error fetching data for chunk 11000 to 12000: 400
Error fetching data for chunk 12000 to 13000: 400
Error fetching data for chunk 13000 to 14000: 400
Error fetching data for chunk 14000 to 15000: 400
Error fetching data for chunk 15000 to 16000: 400
Error fetching data for chunk 16000 to 17000: 400
Error fetching data for chunk 17000 to 18000: 400
Error fetching data for chunk 18000 to 19000: 400
Error fetching data for chunk 19000 to 20000: 400
Error fetching data fo

KeyboardInterrupt: 

In [ ]:
# Step 3: Query Data Using SQL
def query_data_from_db(db_path, query):
    # Connect to the SQLite database
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    
    # Execute the query
    cursor.execute(query)
    
    # Fetch the results and load into a DataFrame
    results = cursor.fetchall()
    columns = [description[0] for description in cursor.description]
    result_df = pd.DataFrame(results, columns=columns)
    
    # Close the connection
    conn.close()
    
    return result_df


In [ ]:
# Define the database path and SQL query
db_path = 'weather_data.db'
query = "SELECT * FROM weather"

# Execute the query and print the results
result_df = query_data_from_db(db_path, query)
print(result_df)
